In [4]:
%pip install tavily-python


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
%run book_writer.py



============ BUSINESS ANALYST ============
[Business Analyst] ```
- 목표: JYP와 HYBE에 관한 책을 집필
- 방법: JYP와 HYBE의 역사, 비즈니스 모델, 성과 및 영향력 등을 조사하여 목차를 작성하고, 각 주제에 대한 세부 정보를 수집하여 내용을 구성한다.
```


============ SUPERVISOR ============
[Supervisor] agent='communicator' done=True description='사용자로부터 JYP와 HYBE에 관한 책의 구체적인 내용을 더 명확히 파악하기 위해 사용자가 원하는 방향이나 주제를 묻고, 진행상황을 확인하여 다음 지시를 받는다.' done_at='2025-04-28T01:03:55Z'


============ COMMUNICATOR ============

AI	: 안녕하세요! 현재 저희 AI 팀은 JYP와 HYBE에 관한 책을 집필하기 위해 기초 조사를 진행 중입니다. 현재까지는 두 엔터테인먼트 회사의 역사, 비즈니스 모델, 성과 및 영향력 등을 중점적으로 조사하고 있습니다. 사용자님께서 이 책에서 특히 다루었으면 하는 주제나 방향이 있다면 말씀해 주세요. 귀하의 의견을 반영하여 책의 내용을 더 풍부하게 만들겠습니다. 추가적인 피드백이나 다른 질문이 있으시면 언제든지 말씀해 주세요!
------------------------------------ MESSAGE COUNT	 5


============ BUSINESS ANALYST ============
[Business Analyst] ```
- 목표: JYP와 HYBE의 비즈니스 모델과 CEO에 대한 심층적 분석을 포함한 책 집필
- 방법: JYP와 HYBE의 비즈니스 모델과 CEO에 대한 조사 및 분석을 통해 목차를 작성하고, 관련 세부 정보를 수집하여 내용을 구성한다. 두 회사의 최신 비즈니스 전략 및 CEO의 리더십 스타일, 경영 철학 등을 분석하여 사용자 요구사

MissingAPIKeyError: No API key provided. Please provide the api_key attribute or set the TAVILY_API_KEY environment variable.